# 06 - Model Deployment

The purpose of this notebook is to execute a CI/CD routine to test and deploy the trained model to `Vertex AI` as an `Endpoint` for online prediction serving. The notebook covers the following steps:
1. Run the test steps locally.
2. Execute the model deployment `CI/CD` steps using `Cloud Build`.



## Setup

### Import libraries

In [10]:
%load_ext autoreload
%autoreload 2

import os
import logging

logging.getLogger().setLevel(logging.INFO)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Setup Google Cloud project

In [11]:
PROJECT = 'pbalm-cxb-aa'
REGION = 'europe-west4'
BUCKET =  PROJECT + '-eu'
SERVICE_ACCOUNT = '188940921537-compute@developer.gserviceaccount.com'

if PROJECT == "" or PROJECT is None or PROJECT == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT = shell_output[0]
    
if SERVICE_ACCOUNT == "" or SERVICE_ACCOUNT is None or SERVICE_ACCOUNT == "[your-service-account]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.account)' 2>/dev/null
    SERVICE_ACCOUNT = shell_output[0]
    
if BUCKET == "" or BUCKET is None or BUCKET == "[your-bucket-name]":
    # Get your bucket name to GCP project id
    BUCKET = PROJECT
    # Try to create the bucket if it doesn't exists
    ! gsutil mb -l $REGION gs://$BUCKET
    print("")
    
print("Project ID:", PROJECT)
print("Region:", REGION)
print("Bucket name:", BUCKET)
print("Service Account:", SERVICE_ACCOUNT)

Project ID: pbalm-cxb-aa
Region: europe-west4
Bucket name: pbalm-cxb-aa-eu
Service Account: 188940921537-compute@developer.gserviceaccount.com


### Set configurations

In [12]:
VERSION = 'v02'
DATASET_DISPLAY_NAME = 'creditcards'
MODEL_DISPLAY_NAME = f'{DATASET_DISPLAY_NAME}-classifier-{VERSION}'
ENDPOINT_DISPLAY_NAME = f'{DATASET_DISPLAY_NAME}-classifier'

CICD_IMAGE_NAME = 'cicd:latest'
CICD_IMAGE_URI = f"gcr.io/{PROJECT}/{CICD_IMAGE_NAME}"

## 1. Run CI/CD steps locally

In [13]:
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION
os.environ['MODEL_DISPLAY_NAME'] = MODEL_DISPLAY_NAME
os.environ['ENDPOINT_DISPLAY_NAME'] = ENDPOINT_DISPLAY_NAME

### Run the model artifact testing

Artifact testing requires that the model is deployed to the Model Registry.

In [15]:
!py.test src/tests/model_deployment_tests.py::test_model_artifact -s

============================= test session starts ==============================
platform linux -- Python 3.7.12, pytest-7.1.2, pluggy-1.0.0
rootdir: /home/jupyter/mlops-with-vertex-ai
plugins: anyio-3.6.1
collected 1 item                                                               

src/tests/model_deployment_tests.py 2022-06-27 12:08:45.065031: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512F
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-27 12:08:45.066424: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
{'V1': <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-0.906611]], dtype=float32)>, 'V2': <tf.Tensor: shape=(1, 1), dtype=flo

### Run create endpoint

In [16]:
!python build/utils.py \
    --mode=create-endpoint\
    --project={PROJECT}\
    --region={REGION}\
    --endpoint-display-name={ENDPOINT_DISPLAY_NAME}

Creating Endpoint
INFO:google.cloud.aiplatform.models:Creating Endpoint
Create Endpoint backing LRO: projects/188940921537/locations/europe-west4/endpoints/831318751528878080/operations/1571563305661628416
INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/188940921537/locations/europe-west4/endpoints/831318751528878080/operations/1571563305661628416
Endpoint created. Resource name: projects/188940921537/locations/europe-west4/endpoints/831318751528878080
INFO:google.cloud.aiplatform.models:Endpoint created. Resource name: projects/188940921537/locations/europe-west4/endpoints/831318751528878080
To use this Endpoint in another session:
INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/188940921537/locations/europe-west4/endpoints/831318751528878080')
INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/188940921537/locations/europe-west4/endpoints/831318751528878080')


### Run deploy model

In [17]:
!python build/utils.py \
    --mode=deploy-model\
    --project={PROJECT}\
    --region={REGION}\
    --endpoint-display-name={ENDPOINT_DISPLAY_NAME}\
    --model-display-name={MODEL_DISPLAY_NAME}

Deploying Model projects/188940921537/locations/europe-west4/models/815274677856370688 to Endpoint : projects/188940921537/locations/europe-west4/endpoints/831318751528878080
INFO:google.cloud.aiplatform.models:Deploying Model projects/188940921537/locations/europe-west4/models/815274677856370688 to Endpoint : projects/188940921537/locations/europe-west4/endpoints/831318751528878080
Deploy Endpoint model backing LRO: projects/188940921537/locations/europe-west4/endpoints/831318751528878080/operations/7974556075875631104
INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/188940921537/locations/europe-west4/endpoints/831318751528878080/operations/7974556075875631104
Endpoint model deployed. Resource name: projects/188940921537/locations/europe-west4/endpoints/831318751528878080
INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/188940921537/locations/europe-west4/endpoints/831318751528878080


### Test deployed model endpoint

In [18]:
!py.test src/tests/model_deployment_tests.py::test_model_endpoint

============================= test session starts ==============================
platform linux -- Python 3.7.12, pytest-7.1.2, pluggy-1.0.0
rootdir: /home/jupyter/mlops-with-vertex-ai
plugins: anyio-3.6.1
collected 1 item                                                               

src/tests/model_deployment_tests.py .                                    [100%]

=============================== warnings summary ===============================
../../../opt/conda/lib/python3.7/site-packages/flatbuffers/compat.py:19
  /opt/conda/lib/python3.7/site-packages/flatbuffers/compat.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
    import imp

../../../opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/utils.py:23
  /opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/utils.py:23: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NE

## 2. Execute the Model Deployment CI/CD routine in Cloud Build

The CI/CD routine is defined in the [model-deployment.yaml](model-deployment.yaml) file, and consists of the following steps:
1. Load and test the the trained model interface.
2. Create and endpoint in Vertex AI if it doesn't exists.
3. Deploy the model to the endpoint.
4. Test the endpoint.

### Build CI/CD container Image for Cloud Build

This is the runtime environment where the steps of testing and deploying model will be executed.

In [19]:
!echo $CICD_IMAGE_URI

gcr.io/pbalm-cxb-aa/cicd:latest


In [20]:
!cp build/Dockerfile.cicd build/Dockerfile
!gcloud builds submit --tag $CICD_IMAGE_URI build/. --timeout=15m

Creating temporary tarball archive of 14 file(s) totalling 27.1 KiB before compression.
Uploading tarball of [build/.] to [gs://pbalm-cxb-aa_cloudbuild/source/1656333986.777081-49d7cb7fcdf045948602fd8482986ffc.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/pbalm-cxb-aa/locations/global/builds/49f074be-7c31-4700-bd28-5d05e642dc59].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/49f074be-7c31-4700-bd28-5d05e642dc59?project=188940921537].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "49f074be-7c31-4700-bd28-5d05e642dc59"

FETCHSOURCE
Fetching storage object: gs://pbalm-cxb-aa_cloudbuild/source/1656333986.777081-49d7cb7fcdf045948602fd8482986ffc.tgz#1656333987302952
Copying gs://pbalm-cxb-aa_cloudbuild/source/1656333986.777081-49d7cb7fcdf045948602fd8482986ffc.tgz#1656333987302952...
/ [1 files][  4.2 KiB/  4.2 KiB]                                                
Operation completed over 1 objects/4.2

### Run CI/CD from model deployment using Cloud Build

In [21]:
#REPO_URL = "https://github.com/GoogleCloudPlatform/mlops-with-vertex-ai.git" # Change to your github repo.
REPO_URL = "https://github.com/pbalm/mlops-with-vertex-ai.git"
BRANCH = "main" 

In [22]:
SUBSTITUTIONS=f"""\
_REPO_URL='{REPO_URL}',\
_BRANCH={BRANCH},\
_CICD_IMAGE_URI={CICD_IMAGE_URI},\
_PROJECT={PROJECT},\
_REGION={REGION},\
_MODEL_DISPLAY_NAME={MODEL_DISPLAY_NAME},\
_ENDPOINT_DISPLAY_NAME={ENDPOINT_DISPLAY_NAME},\
_GCS_BUCKET={BUCKET}/cloudbuild,\
_SERVICE_ACCOUNT={SERVICE_ACCOUNT}\
"""

!echo $SUBSTITUTIONS

_REPO_URL=https://github.com/pbalm/mlops-with-vertex-ai.git,_BRANCH=main,_CICD_IMAGE_URI=gcr.io/pbalm-cxb-aa/cicd:latest,_PROJECT=pbalm-cxb-aa,_REGION=europe-west4,_MODEL_DISPLAY_NAME=creditcards-classifier-v02,_ENDPOINT_DISPLAY_NAME=creditcards-classifier,_GCS_BUCKET=pbalm-cxb-aa-eu/cloudbuild,_SERVICE_ACCOUNT=188940921537-compute@developer.gserviceaccount.com


In [23]:
!gcloud builds submit --no-source --config build/model-deployment.yaml --substitutions {SUBSTITUTIONS} --timeout=30m

Created [https://cloudbuild.googleapis.com/v1/projects/pbalm-cxb-aa/locations/global/builds/a00cb9fa-9371-4d50-9c58-85f2e106ee2c].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/a00cb9fa-9371-4d50-9c58-85f2e106ee2c?project=188940921537].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "a00cb9fa-9371-4d50-9c58-85f2e106ee2c"

FETCHSOURCE
BUILD
Starting Step #0 - "Clone Repository"
Step #0 - "Clone Repository": Already have image (with digest): gcr.io/cloud-builders/git
Step #0 - "Clone Repository": Cloning into 'mlops-with-vertex-ai'...
Step #0 - "Clone Repository": POST git-upload-pack (352 bytes)
Step #0 - "Clone Repository": POST git-upload-pack (194 bytes)
Finished Step #0 - "Clone Repository"
Starting Step #1 - "Test Model Artifact"
Step #1 - "Test Model Artifact": Pulling image: gcr.io/pbalm-cxb-aa/cicd:latest
Step #1 - "Test Model Artifact": latest: Pulling from pbalm-cxb-aa/cicd
Step #1 - "Test Model Arti